## Initial Setup
*Setup input and output object stores
*Get execution role to have security access to AWS account

In [5]:
from sagemaker import get_execution_role

#Bucket location to save custom code
custom_code_upload_location = 's3://kb-sagemaker/customcode/mxnet'

#Bucket location where results of model training are saved.
model_artifacts_location = 's3://kb-sagemaker/artifacts'

#IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
#We can use the Amazon SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

# Setup Estimator
Estimator server for training

In [6]:
from sagemaker.mxnet import MXNet

mnist_estimator = MXNet(entry_point='/home/ec2-user/sample-notebooks/sagemaker-python-sdk/mxnet_mnist/mnist.py',
                        role=role,
                        output_path=model_artifacts_location,
                        code_location=custom_code_upload_location,
                        train_instance_count=1, 
                        train_instance_type='ml.m4.xlarge',
                        hyperparameters={'learning_rate': 0.1})

# Train Model
Load test data an train model deployed from previous step

In [7]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

mnist_estimator.fit({'train': train_data_location, 'test': test_data_location})

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-11-08-18-12-32-532


2018-11-08 18:12:32 Starting - Starting the training job...
2018-11-08 18:12:33 Starting - Launching requested ML instances......
2018-11-08 18:13:36 Starting - Preparing the instances for training......
2018-11-08 18:15:00 Downloading - Downloading input data..
2018-11-08 18:15:11,425 INFO - root - running container entrypoint
2018-11-08 18:15:11,425 INFO - root - starting train task
2018-11-08 18:15:11,431 INFO - container_support.training - Training starting
2018-11-08 18:15:12,259 WARNING - mxnet_container.train - #033This required structure for training scripts will be deprecated with the next major release of MXNet images. The train() function will no longer be required; instead the training script must be able to be run as a standalone script. For more information, see https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/mxnet#updating-your-mxnet-training-script.#033
2018-11-08 18:15:13,676 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudw


2018-11-08 18:15:11 Training - Training image download completed. Training in progress.2018-11-08 18:15:22,109 INFO - root - Epoch[6] Batch [400]#011Speed: 69968.74 samples/sec#011accuracy=0.966400
2018-11-08 18:15:22,261 INFO - root - Epoch[6] Batch [500]#011Speed: 65800.75 samples/sec#011accuracy=0.967200
2018-11-08 18:15:22,406 INFO - root - Epoch[6] Train-accuracy=0.964040
2018-11-08 18:15:22,407 INFO - root - Epoch[6] Time cost=0.878
2018-11-08 18:15:22,491 INFO - root - Epoch[6] Validation-accuracy=0.960500
2018-11-08 18:15:22,648 INFO - root - Epoch[7] Batch [100]#011Speed: 64265.35 samples/sec#011accuracy=0.967426
2018-11-08 18:15:22,797 INFO - root - Epoch[7] Batch [200]#011Speed: 67063.69 samples/sec#011accuracy=0.968600
2018-11-08 18:15:22,941 INFO - root - Epoch[7] Batch [300]#011Speed: 69626.68 samples/sec#011accuracy=0.969800
2018-11-08 18:15:23,094 INFO - root - Epoch[7] Batch [400]#011Speed: 65424.47 samples/sec#011accuracy=0.971800
2018-11-08 18:15:23,254 INFO - root 

2018-11-08 18:15:32,084 INFO - root - Epoch[16] Batch [200]#011Speed: 69319.78 samples/sec#011accuracy=0.990900
2018-11-08 18:15:32,233 INFO - root - Epoch[16] Batch [300]#011Speed: 67422.78 samples/sec#011accuracy=0.991500
2018-11-08 18:15:32,392 INFO - root - Epoch[16] Batch [400]#011Speed: 62923.97 samples/sec#011accuracy=0.991200
2018-11-08 18:15:32,547 INFO - root - Epoch[16] Batch [500]#011Speed: 64494.12 samples/sec#011accuracy=0.992600
2018-11-08 18:15:32,703 INFO - root - Epoch[16] Train-accuracy=0.991414
2018-11-08 18:15:32,703 INFO - root - Epoch[16] Time cost=0.941
2018-11-08 18:15:32,792 INFO - root - Epoch[16] Validation-accuracy=0.971000
2018-11-08 18:15:32,960 INFO - root - Epoch[17] Batch [100]#011Speed: 60119.00 samples/sec#011accuracy=0.992079
2018-11-08 18:15:33,117 INFO - root - Epoch[17] Batch [200]#011Speed: 63609.62 samples/sec#011accuracy=0.992300
2018-11-08 18:15:33,275 INFO - root - Epoch[17] Batch [300]#011Speed: 63470.24 samples/sec#011accuracy=0.992500
201

# Deploy Trained Model to Endpoint for Predictions
Create server instance to host trained model and act as an endpoint for prediction API calls

In [8]:
%%time

predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-11-08-18-12-32-532
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-11-08-18-12-32-532


--------------------------------------------------------------!CPU times: user 300 ms, sys: 0 ns, total: 300 ms
Wall time: 5min 13s


# Setup HTML Drawing Client

In [47]:
from IPython.display import HTML
HTML(open("/home/ec2-user/sample-notebooks/sagemaker-python-sdk/mxnet_mnist/input.html").read())

# Make inference for above drawing

In [58]:
response = predictor.predict(data)
print('Raw prediction result:')
print(response)

labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

Raw prediction result:
[[1.162921433548467e-14, 0.0004702737496700138, 0.9995296001434326, 6.82744314417505e-08, 5.473607373287278e-16, 9.93337665980865e-16, 1.7941066384266624e-12, 2.0634491448845438e-08, 9.843352799521199e-09, 2.5989752216571644e-18]]
Labeled predictions: 
[(0, 1.162921433548467e-14), (1, 0.0004702737496700138), (2, 0.9995296001434326), (3, 6.82744314417505e-08), (4, 5.473607373287278e-16), (5, 9.93337665980865e-16), (6, 1.7941066384266624e-12), (7, 2.0634491448845438e-08), (8, 9.843352799521199e-09), (9, 2.5989752216571644e-18)]
Most likely answer: (2, 0.9995296001434326)


In [59]:
print(data)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 